# ORDER TO RUN

ValidPixels

CroppingData

Training

FullMap

Correcting

In [1]:
import numpy as np
from numpy import savetxt
from numpy import loadtxt
import fitsio
import healpy as hp
import matplotlib.pyplot as plt
import Config

In [2]:
res = 4096 # Resolution of the heal pixels
numBins = 100 # Number of bins in fitting
perVar = 0.98 # Percent of the variance to be captured
perMap = 0.625 # Percent of the PC maps to use, adjust this later

In [3]:
condMean = []
condStds = []

In [4]:
validPix = fitsio.read("/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Valid_4096_Pixels.fits")['PIXEL']
pixCheck = np.full(12*(4096**2), False, dtype = bool)
pixCheck[validPix] = True

In [5]:
# This is the actual file containing all of the data
balrFile = '/afs/hep.wisc.edu/bechtol-group/MegansThings/balrog_detection_catalog_sof_run2_stars_v1.4_avg_added_match_flags.fits'
# This reads in all of the data. Most of these are just flags, the only pieces that get used much outside
# of filtering are detected, true_ra and true_dec which get used to convert into healPixels.
balrData = fitsio.read(balrFile, columns = ['detected', 'true_ra', 'true_dec',
                                            'flags_foreground', 'flags_badregions', 'flags_footprint',
                                            'match_flag_1.5_asec'])

# These are in degrees which is why lonlat is set to True in the next cell.
balrRA = balrData['true_ra']
balrDEC = balrData['true_dec']
# This is used for detection rates, each point is either a 0 (no detection) or a 1 (detection)
balrDETRepeats = balrData['detected']
# Everything from here on out is simply used in order to filter the data
FOREGROUND = balrData['flags_foreground']
BADREGIONS = balrData['flags_badregions']
FOOTPRINT = balrData['flags_footprint']
ARCSECONDS = balrData['match_flag_1.5_asec']

# This is used to filter out any injections that either weren't detected or had flags raised.
cutIndices = np.where((FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      (ARCSECONDS < 2))[0]

# This reduced the data down to the actually valid pixels.
balrDETRepeats = balrDETRepeats[cutIndices]
balrRA = balrRA[cutIndices]
balrDEC = balrDEC[cutIndices]

# This converts the RA and DEC values from above to healpixels so we can compare to the sky condition.
balrPIXRepeats = hp.ang2pix(res, balrRA, balrDEC, lonlat = True, nest = True)

# This sorts by the pixel in order to make following methods more efficient.
sortInds = balrPIXRepeats.argsort()
balrPIXRepeats = balrPIXRepeats[sortInds[::1]]
balrDETRepeats = balrDETRepeats[sortInds[::1]]

# These are indices that will be looping through the pixStar and starPix arrays in parallel.
uniqInd = 0
balrInd = 0

# This will be used to store the number of stars at each pixel.
balrPIX = np.unique(balrPIXRepeats) # The unique pixels, with no repeats.
balrDET = np.zeros_like(balrPIX)
balrINJ = np.zeros_like(balrPIX)

while balrInd < len(balrPIXRepeats):
    if balrPIX[uniqInd] == balrPIXRepeats[balrInd]: # If the pixels match up in the arrays.
        balrDET[uniqInd] += balrDETRepeats[balrInd] # Add one if there was a detection at this location.
        balrINJ[uniqInd] += 1                # Add one to the corresponding spot in the balStar array.
        balrInd += 1                         # Add one to the starInd to see if the next index in starPix is also the same.
        # Since the last index of pixStar and starPix are the same, starInd will increase the last time through the loop,
        # making this the index that we must restrict in the while loop.
    else:
        uniqInd += 1 # If the pixels are no longer the same, increase the index you check in the pixStar array.
        
balrDET = balrDET[pixCheck[balrPIX]]
balrINJ = balrINJ[pixCheck[balrPIX]]
balrPIX = balrPIX[pixCheck[balrPIX]]

In [6]:
# This loads in all of the file names of the survey conditions
directory = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/'
conditions = Config.conditions
balrCondMaps = []
# fullCondMaps = []

# This loops over every condition file except for stellar density which has a different format
for cond in conditions:
    condData = fitsio.read(directory + cond + '.fits') # This reads in the data
    condSigExt = np.full(12*(4096**2), -1.6375e+30) # Gives a default value
    condSigExt[validPix] = condData['SIGNAL'] # Changes all valid pixels to their corresponding signals
    condSigExt[np.where(condSigExt == -1.6375e+30)[0]] = hp.UNSEEN # Masks all non valid pixels
    if res != 4096:
        condSigExt=hp.ud_grade(condSigExt, res) # Only degrades if necessary (this is very time consuming)
    balrCondMaps.append(condSigExt[balrPIX]) # Only stores the values that are in pixels with injections

balrCondMaps = np.array(balrCondMaps, dtype = object) # Converts to an array

# Stores the original data for later comparisons
originalBalrDET = balrDET
originalBalrINJ = balrINJ
aveEff = np.sum(originalBalrDET) / np.sum(originalBalrINJ)

In [7]:
balrStanMaps = []
# This standardizes every map as a first step of PCA
for i in range(len(balrCondMaps)):
    condMean.append(np.average(balrCondMaps[i]))
    condStds.append(np.std(balrCondMaps[i]))
    balrStanMaps.append((balrCondMaps[i] - np.average(balrCondMaps[i])) / np.std(balrCondMaps[i]))
    
    
balrStanMaps = np.array(balrStanMaps)

In [8]:
# This gives the covariance matrix of the standardized maps
# Bias is true since the variance of each individual standardized map should be 1
cov = np.cov(balrStanMaps.astype(float), bias = True)

# This gives the eigenvalues and vectors of the covariance matrix
evalues, evectors = np.linalg.eig(cov)

# This cuts after the specified percentage of the variance has been achieved
for i in range(len(evalues)):
    if np.sum(evalues[0:i+1]) / np.sum(evalues) >= perVar:
        cutoff = i + 1
        break
featVec = evectors[0:cutoff]

In [9]:
balrRedMaps = np.matmul(featVec, balrStanMaps) # Reduces the maps to PCA maps

In [10]:
# Goal of this method is to find the index of the map that has the largest impact on detection rates.
def mostSigPCMap(redMaps, balrDET, balrINJ = balrINJ, numBins = numBins):
    
    maxAdjustment = []

    for i in range(len(redMaps)):
        
        onePC = redMaps[i] # Load up a PC map

        binIndLims = [0]

        for j in range(numBins):
            binIndLims.append(int((len(onePC) - binIndLims[-1]) / (numBins - j)) + (binIndLims[-1]))

        # Sort detections and injections by the PC map value.
        sortInds = onePC.argsort()
        balrDETTemp = balrDET[sortInds[::1]]
        balrINJTemp = balrINJ[sortInds[::1]]

        detBins = []
        injBins = []

        # Bin up these detections and injections.
        for j in range(numBins):
            detBins.append(balrDETTemp[binIndLims[j]:binIndLims[j+1]])
            injBins.append(balrINJTemp[binIndLims[j]:binIndLims[j+1]])

        yValues = []

        # For each bin find the detection rate relative to the average.
        for j in range(numBins):
            yValues.append((np.sum(detBins[j]) / np.sum(injBins[j])) / aveEff)

        yValues = np.array(yValues)
        
        # Make the error the sum of the squared difference between the binned values and 1.
        maxAdjustment.append(np.sum((yValues - 1)**2))
        
    maxAdjustment = np.array(maxAdjustment)
    
    mostSigIndex = np.where(maxAdjustment == np.max(maxAdjustment))[0]
    
    return mostSigIndex[0] # Return wherever the error is the largest

In [11]:
# Goal is to find the best degree for fitting the data between one and five.
def findBestDegree(index, redMaps, balrDET, balrINJ = balrINJ, numBins = numBins):
    
    onePC = redMaps[index] # Load up a PC map
    
    # Gives the errors at each fit
    errors = []
    
    for potDegrees in (np.arange(5) + 1): # Potential degrees between 1 and 5
        
        balrDETTemp = np.copy(balrDET)
        
        fitParams = getFitParams(index, redMaps, potDegrees, balrDET)
        
        correction = np.full(len(balrDET), 0.0)
            
        for i in range(len(fitParams)):
            correction = correction + (fitParams[i] * (redMaps[index]**(len(fitParams) - 1 - i)))
                
        correction = 1 / correction
        
        balrDETTemp = balrDETTemp * correction # Apply the correction at the certain degree
        
        onePC = redMaps[index]
    
        binIndLims = [0]

        for j in range(numBins):
            binIndLims.append(int((len(onePC) - binIndLims[-1]) / (numBins - j)) + (binIndLims[-1]))

        sortInds = onePC.argsort()
        balrDETTemp = balrDETTemp[sortInds[::1]]
        balrINJTemp = balrINJ[sortInds[::1]]

        detBins = []
        injBins = []

        for j in range(numBins):
            detBins.append(balrDETTemp[binIndLims[j]:binIndLims[j+1]])
            injBins.append(balrINJTemp[binIndLims[j]:binIndLims[j+1]])

        yValues = []

        for j in range(numBins):
            yValues.append((np.sum(detBins[j]) / np.sum(injBins[j])) / aveEff)

        yValues = np.array(yValues)
        
        errors.append(np.sum((yValues - 1)**2)) # See how far off the calculated y Values are
        
    errors = np.array(errors)
        
    return (np.where(errors == np.min(errors))[0] + 1)

In [12]:
def getFitParams(index, redMaps, fitDegree, balrDET, balrINJ = balrINJ, numBins = numBins):
    
    onePC = redMaps[index] # This loads up the PC Map we'll be using
    
    binIndLims = [0]

    for j in range(numBins):
        binIndLims.append(int((len(onePC) - binIndLims[-1]) / (numBins - j)) + (binIndLims[-1]))
        
    # This gives the limits (in index) of the bins

    # Sorts the data according to the value of the PC map
    sortInds = onePC.argsort()
    onePC = onePC[sortInds[::1]]
    balrDETTemp = balrDET[sortInds[::1]]
    balrINJTemp = balrINJ[sortInds[::1]]

    detBins = []
    injBins = []
    pcBins = []
    
    # Generates the bins
    for j in range(numBins):
        detBins.append(balrDETTemp[binIndLims[j]:binIndLims[j+1]])
        injBins.append(balrINJTemp[binIndLims[j]:binIndLims[j+1]])
        pcBins.append(onePC[binIndLims[j]:binIndLims[j+1]])

    yValues = []
    xValues = []

    # The x values are a weighted average of the PC values with weights in terms of how many injections were on that pixel
    # The y values are the detection rate relative to theaverage efficiency
    for j in range(numBins):
        yValues.append((np.sum(detBins[j]) / np.sum(injBins[j])) / aveEff)
        xValues.append(np.sum(pcBins[j] * injBins[j]) / np.sum(injBins[j]))

    xValues = np.array(xValues)
    yValues = np.array(yValues)
    
    # This fits the data to a polynomial of specified degree
    p = np.polyfit(xValues, yValues, fitDegree)
    
    return p

In [13]:
# This method does the same thing as the one above it but doesn't plot out a fit line.
# As well as this, no parameters are returned since no fit was performed.
def plotNoFit(index, redMaps, balrDET, size = 100, balrINJ = balrINJ, numBins = numBins):
    
    onePC = redMaps[index]
    
    binIndLims = [0]

    for j in range(numBins):
        binIndLims.append(int((len(onePC) - binIndLims[-1]) / (numBins - j)) + (binIndLims[-1]))

    sortInds = onePC.argsort()
    onePC = onePC[sortInds[::1]]
    detStarTemp = balrDET[sortInds[::1]]
    injStarTemp = balrINJ[sortInds[::1]]

    detBins = []
    injBins = []
    pcBins = []
    
    for j in range(numBins):
        detBins.append(detStarTemp[binIndLims[j]:binIndLims[j+1]])
        injBins.append(injStarTemp[binIndLims[j]:binIndLims[j+1]])
        pcBins.append(onePC[binIndLims[j]:binIndLims[j+1]])

    yValues = []
    xValues = []

    for j in range(numBins):
        yValues.append((np.sum(detBins[j]) / np.sum(injBins[j])) / aveEff)
        xValues.append(np.sum(pcBins[j] * injBins[j]) / np.sum(injBins[j]))

    xValues = np.array(xValues)
    yValues = np.array(yValues)

    plt.figure(dpi = size)
    plt.plot(xValues, yValues, marker = '.', ms = 10, zorder = 4)
    plt.axhline(y = 1,color = 'k', linestyle = '--', zorder = 2)
    plt.xlabel('PC Value')
    plt.ylabel('N/⟨N⟩')
    plt.title('Detection Rate vs PC Map Value')
    plt.grid(zorder = 0)
    plt.show()

In [14]:
def plotFit(index, redMaps, fitDegree, balrDET, size = 100, balrINJ = balrINJ, numBins = numBins):
    
    onePC = redMaps[index]
    
    binIndLims = [0]

    for j in range(numBins):
        binIndLims.append(int((len(onePC) - binIndLims[-1]) / (numBins - j)) + (binIndLims[-1]))

    sortInds = onePC.argsort()
    onePC = onePC[sortInds[::1]]
    detStarTemp = balrDET[sortInds[::1]]
    injStarTemp = balrINJ[sortInds[::1]]

    detBins = []
    injBins = []
    pcBins = []
    
    for j in range(numBins):
        detBins.append(detStarTemp[binIndLims[j]:binIndLims[j+1]])
        injBins.append(injStarTemp[binIndLims[j]:binIndLims[j+1]])
        pcBins.append(onePC[binIndLims[j]:binIndLims[j+1]])

    yValues = []
    xValues = []

    for j in range(numBins):
        yValues.append((np.sum(detBins[j]) / np.sum(injBins[j])) / aveEff)
        xValues.append(np.sum(pcBins[j] * injBins[j]) / np.sum(injBins[j]))

    xValues = np.array(xValues)
    yValues = np.array(yValues)
    
    p = np.polyfit(xValues, yValues, fitDegree)
    # Up until this point, the method is identical to the one above
    
    yFitValues = np.zeros_like(xValues)
    
    # Here we generate what the fit values are going to be
    for i in range(len(p)):
        yFitValues += p[i] * (xValues**(len(p) - 1 - i))

    # This plots out the data and fit line
    plt.figure(dpi = size)
    plt.plot(xValues, yValues, marker = '.', ms = 10, zorder = 4, label = 'Data')
    plt.axhline(y = 1,color = 'k', linestyle = '--', zorder = 2)
    plt.plot(xValues, yFitValues, zorder = 3, label = 'Fit')
    plt.legend()
    plt.xlabel('PC Value')
    plt.ylabel('N/⟨N⟩')
    plt.title('Degree ' + str(len(p) - 1) + ' Fit, Index ' + str(index))
    plt.grid(zorder = 0)
    plt.show()
    
    # This returns the fit parameters
    return p

In [15]:
balrDET = originalBalrDET
corrIndices = []
corrFitParams = []

In [16]:
# CORRECTIONS = np.ones(len(PIX_4096))

In [17]:
trimBalrRedMaps = np.copy(balrRedMaps)
# trimFullRedMaps = np.copy(FullRedMaps)

iterations = int(perMap * len(balrRedMaps))

for _ in range(iterations):

    index = mostSigPCMap(trimBalrRedMaps, balrDET)
    
    corrIndices.append(index)

    degree = findBestDegree(index, trimBalrRedMaps, balrDET)

    # plotFit(index, trimBalrRedMaps, degree, balrDET)

    fitParams = getFitParams(index, trimBalrRedMaps, degree, balrDET)
    
    corrFitParams.append(fitParams)

    balrCorr = np.full(len(balrDET), 0.0)

    for i in range(len(fitParams)):
        balrCorr = balrCorr + (fitParams[i] * (trimBalrRedMaps[index]**(len(fitParams) - 1 - i)))

    balrCorr = 1 / balrCorr

    balrDET = balrDET * balrCorr

    # plotNoFit(index, trimBalrRedMaps, balrDET)

    # fullCorr = np.full(len(CORRECTIONS), 0.0)

    # for i in range(len(fitParams)):
    #     fullCorr = fullCorr + (fitParams[i] * (trimFullRedMaps[index]**(len(fitParams) - 1 - i)))

    # fullCorr = 1 / fullCorr

    # CORRECTIONS = CORRECTIONS * fullCorr
    
    pcMapCutoff = np.full(len(trimBalrRedMaps), True, dtype = bool)
    pcMapCutoff[index] = False
    trimBalrRedMaps = trimBalrRedMaps[pcMapCutoff]

In [18]:
actualCorrIndices = []
originalIndices = np.arange(len(balrRedMaps))

for index in corrIndices:
    actualCorrIndices.append(originalIndices[index])
    originalIndices = np.delete(originalIndices, index)
    
actualCorrIndices = np.array(actualCorrIndices)
print(len(actualCorrIndices))
print(len(np.unique(actualCorrIndices)))

30
30


In [19]:
testBalrDET = originalBalrDET

for i in range(len(actualCorrIndices)):
    
    balrCorr = np.full(len(testBalrDET), 0.0)
    
    for j in range(len(corrFitParams[i])):
        balrCorr = balrCorr + (corrFitParams[i][j] * (balrRedMaps[actualCorrIndices[i]]**(len(corrFitParams[i]) - 1 - j)))
        
    balrCorr = 1 / balrCorr

    testBalrDET = testBalrDET * balrCorr

In [20]:
corrFitParams = np.array(corrFitParams, dtype = object)

In [21]:
actualCorrFitParams = []

for parameters in corrFitParams:
    actualCorrFitParams.append(list(parameters))
    
actualCorrFitParams = np.array(actualCorrFitParams, dtype = object)

In [22]:
len(condMean)

92

In [23]:
savetxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Means' + str(numBins) + '.csv', condMean, delimiter=',')

In [24]:
savetxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Standard_Deviations' + str(numBins) + '.csv', condStds, delimiter=',')

In [25]:
savetxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Indices' + str(numBins) + '.csv', actualCorrIndices, delimiter=',')

In [26]:
savetxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Fit_Parameters' + str(numBins) + '.csv', actualCorrFitParams, delimiter='\t', fmt='%s')

In [27]:
test = loadtxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Fit_Parameters' + str(numBins) + '.csv', delimiter = '\t', dtype = object)

In [28]:
conToArray = test[0]

In [29]:
conToArray = conToArray[1:len(conToArray) - 1]

In [30]:
conToArray

'-9.324767046645395e-05, -0.0005113468521127358, 0.0030724565988606302, -0.0011046211079423353, 0.022063290051698086, 1.0035955948761426'

In [31]:
float(conToArray.split(', ')[0])

-9.324767046645395e-05

In [32]:
featVec

array([[ 0.05705898, -0.19186478,  0.08102498, ..., -0.00932484,
        -0.02231058, -0.00233181],
       [ 0.04778338, -0.21311289,  0.07186065, ...,  0.04766   ,
        -0.0112544 , -0.00703989],
       [ 0.04619177, -0.22162086,  0.08329973, ..., -0.04620647,
         0.0076669 ,  0.01655152],
       ...,
       [ 0.04020286,  0.16698352,  0.05507521, ...,  0.043096  ,
        -0.00212672, -0.03468285],
       [ 0.05448686,  0.07972472,  0.00935943, ..., -0.10243191,
         0.02759082, -0.06562796],
       [-0.13259782,  0.01085495,  0.1805411 , ..., -0.09978466,
        -0.04357347,  0.13032148]])

In [33]:
savetxt('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/Feature_Vectors' + str(numBins) + '.csv', featVec, delimiter='\t', fmt='%s')